<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=280113860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Library importing

In [9]:
!pip install xgboost
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import gc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/depi-dataset/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [3]:
del chunks

In [4]:
preprocessor=joblib.load("/kaggle/input/depi-dataset/pre-proccess.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

In [22]:
categorical_cols = []
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "sell_price","lag_1",'price_flag','lag_7','snap_weekend','wday_x_snap','is_weekend','event_impact','event_count','is_event']
categorical_cols += ["item_category", "item_subcategory",'item_number']
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])
joblib.dump(preprocessor,"/kaggle/working/pre-proccess.pkl")

## Data spliting

In [23]:
store_models = {}

stores = final_data["store_id"].unique()

for store in stores:
    print(f"\n==== Training model for store: {store} ====")

    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X.drop(columns='store_id',inplace=True)
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )

    # 4️⃣ Fit model
    
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose = False
    )
    model.score(X_train,y_train) , model.score(X_test,y_test)
    from sklearn.metrics import mean_absolute_error, r2_score

    train_mae = mean_absolute_error(y_train, model.predict(X_train))
    test_mae  = mean_absolute_error(y_test,  model.predict(X_test))
    train_r2  = r2_score(y_train, model.predict(X_train))
    test_r2   = r2_score(y_test,  model.predict(X_test))

    print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
    print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")
   
    # 5️⃣ Save model
    store_models[store] = model
    print(f"Model for {store} trained and stored.")

    del X, y, X_temp, X_test, X_val, X_train, y_train, y_temp, y_test
    gc.collect()


==== Training model for store: CA_1 ====
MAE train: 0.3910, test: 0.4155
R2  train: 0.9241, test: 0.8843
Model for CA_1 trained and stored.

==== Training model for store: CA_2 ====
MAE train: 0.3156, test: 0.3349
R2  train: 0.8887, test: 0.8463
Model for CA_2 trained and stored.

==== Training model for store: CA_3 ====


KeyboardInterrupt: 

In [ ]:
import numpy as np
for store in stores:
    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X.drop(columns='store_id',inplace=True)
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )
    y_pred = store_models[store].predict(X_test)
    # Round predictions and ground truth
    y_pred_rounded = np.round(y_pred).astype(int)
    y_test_rounded = np.round(y_test).astype(int)

    print("\n=== Sample Predictions vs Actuals ===\n")
    for i in range(min(25, len(y_test))):
        print(f"Pred: {y_pred_rounded[i]:>3}   Actual: {y_test_rounded.iloc[i]:>3}")

In [ ]:
for i in store_models.keys():
    joblib.dump(store_models[i], f"/kaggle/working/{i}.pkl")